In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import torch
import torchvision.transforms as transforms
from globsML.utils.eval import get_test_metrics
from globsML.utils.draw import plot_source
from globsML.utils.imageloader import load_data, CustomGCDataset
from globsML.utils.training import train_CNN as train
from globsML.utils.training import test_CNN as test
from globsML.models.CNN import CNN
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm, trange

# list of thresholds used to calculate AUC ROC
thresh = np.arange(0,1.02,0.01)

# data splits
BATCHSIZE = 500
TEST_SIZE = 0.2
EVAL_SIZE = 0.05

### Train CNN on different random data splits

In [3]:
# create random seeds for data splits
np.random.seed(123124)
seeds = list(map(int, np.random.random(10)*21321456))

# do the experiment for each random split
for SEED in seeds:
    # load tabular data
    data_path = '../data/ACS_sources_original.csv'
    data = pandas.read_csv(data_path)
    galaxies_to_test = set([])

    # load image data
    available_galaxies = set(data['galaxy'].unique())
    available_galaxies = available_galaxies.difference(set(['VCC538']))
    images, labels, probabilities, galaxies, IDs = load_data(data, available_galaxies)

    # set random seeds
    np.random.seed(SEED)
    torch.manual_seed(SEED)

    # data splits
    train_images, test_images, train_labels, test_labels = train_test_split(images, probabilities, test_size=TEST_SIZE, random_state=SEED)
    train_images, eval_images, train_labels, eval_labels = train_test_split(train_images, train_labels, test_size=EVAL_SIZE, random_state=SEED+50)

    # set up data loaders
    train_labels[train_labels<=0.2] = 0.
    train_labels[train_labels>=0.8] = 1.
    trainData = CustomGCDataset(train_images, train_labels, 
                            transforms.Compose([transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip()]))

    train_loader = torch.utils.data.DataLoader(trainData, batch_size=BATCHSIZE, shuffle=True)

    eval_labels = np.array(eval_labels>=0.5, dtype=int)
    evalData = CustomGCDataset(eval_images, eval_labels)
    eval_loader = torch.utils.data.DataLoader(evalData, batch_size=BATCHSIZE, shuffle=False)

    test_labels = np.array(test_labels>=0.5, dtype=int)
    testData = CustomGCDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(testData, batch_size=BATCHSIZE, shuffle=False)

    # create CNN
    net = CNN()
    net.cuda()

    # set up optimiser for training
    LR = 1e-3
    REG = 1e-5
    optimizer = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=REG)

    # training loop!
    training_losses = []
    val_tpr = []
    val_fdr = []
    EPOCHS = 100

    best_metric = 0

    for epoch in trange(EPOCHS):
        training_loss, mean_loss = train(net, 'cuda', train_loader, optimizer)
        tpr, fdr = test(net, 'cuda', eval_loader, eval_labels, epoch)
        training_losses += training_loss
        val_tpr.append(tpr)
        val_fdr.append(fdr)
        print('Epoch {} ---- TPR: {} -- FDR: {}'.format(epoch, tpr, fdr))
        if tpr-fdr > best_metric:
            torch.save({
                'epoch': epoch,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'tpr': tpr,
                'fdr': fdr,
                }, 'All2All--CNN--{}'.format(SEED))
            best_metric = tpr-fdr
            print('NEW BEST')

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0 ---- TPR: 0.8650137741046832 -- FDR: 0.2294478527607362
NEW BEST


  0%|          | 0/129 [00:00<?, ?it/s]

...

Epoch 97 ---- TPR: 0.9445910290237467 -- FDR: 0.06892067620286085


  0%|          | 0/129 [00:00<?, ?it/s]

Epoch 98 ---- TPR: 0.9445910290237467 -- FDR: 0.06770833333333333


  0%|          | 0/129 [00:00<?, ?it/s]

Epoch 99 ---- TPR: 0.9261213720316622 -- FDR: 0.056451612903225805


### Evaluate the models on their corresponding training split

In [26]:
np.random.seed(123124)
seeds = list(map(int, np.random.random(10)*21321456))

final_res = pandas.DataFrame()
for SEED in seeds:
    # load model with best validation performance for random seed SEED
    loaded_net = CNN()
    checkpoint = torch.load('All2All--CNN--{}'.format(SEED))
    loaded_net.load_state_dict(checkpoint['model_state_dict'])
    loaded_net.cuda()
    
    # load data again, with same split as during training
    data_path = '../data/ACS_sources_original.csv'
    data = pandas.read_csv(data_path)
    galaxies_to_test = set([])

    available_galaxies = set(data['galaxy'].unique())
    available_galaxies = available_galaxies.difference(set(['VCC538']))
    images, labels, probabilities, galaxies, IDs = load_data(data, available_galaxies)

    np.random.seed(SEED)
    torch.manual_seed(SEED)

    train_images, test_images, train_labels, test_labels = train_test_split(images, probabilities, test_size=TEST_SIZE, random_state=SEED)
    test_labels = np.array(test_labels>=0.5, dtype=int)
    testData = CustomGCDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(testData, batch_size=BATCHSIZE, shuffle=False)

    # test model
    # (with dropout on. Dropout can be deactivated using loaded_net.eval() instead.
    # In this case, the loop is not required and a single call for each test source is sufficient.)
    loaded_net.train()
    res = []

    for i in range(200):
        pred = []
        for x,y in test_loader:
            xout, _ = loaded_net(x.cuda())
            xout = xout.flatten().cpu().detach().numpy()
            pred += list(xout)
        pred = np.array(pred)
        res.append(pred)
    
    nmean = np.mean(res, axis=0)
    nstd = np.std(res,axis=0)
    
    np.save('All2All--mean--{}'.format(SEED), nmean)
    np.save('All2All--std--{}'.format(SEED), nstd)
    
    stats_gal, stats_all, _, _, _, _, _ = get_test_metrics(list(galaxies_to_test), [], [], test_labels, 1.*(nmean>0.5), probs=nmean, thresh=thresh)
    final_res = final_res.append(stats_all)
final_res['seed'] = seeds 
final_res.to_csv('All2All--results')

/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [28]:
final_res.mean()

/tmp/ipykernel_526/3041093142.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_res.mean()


TPR             9.292912e-01
FDR             6.781520e-02
FPR             1.925290e-02
AUC(FDR,TPR)    9.751750e-01
AUC(FPR,TPR)    9.936198e-01
# found GCs     3.467400e+03
# total GCs     3.731300e+03
# fake GCs      2.530000e+02
# sources       1.687000e+04
seed            6.988626e+06
dtype: float64

In [30]:
final_res

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources,seed
0,ALL,0.912650,0.058516,0.016241,0.973388,0.992388,3427,3755,213,16870,2780441
0,ALL,0.929220,0.065036,0.018437,0.974994,0.993457,3479,3744,242,16870,6488346
0,ALL,0.935589,0.073687,0.020873,0.976135,0.994428,3457,3695,275,16870,12269029
0,ALL,0.926045,0.057026,0.015908,0.975530,0.993475,3456,3732,209,16870,2166833
0,ALL,0.916071,0.053093,0.014817,0.976935,0.994177,3460,3777,194,16870,6621300
0,ALL,0.937568,0.083910,0.023860,0.971541,0.993146,3439,3668,315,16870,13698722
0,ALL,0.958356,0.078779,0.023392,0.978145,0.994594,3590,3746,307,16870,2249589
0,ALL,0.943336,0.079917,0.023492,0.976620,0.993890,3546,3759,308,16870,9815943
0,ALL,0.911559,0.066355,0.018327,0.972552,0.992841,3391,3720,241,16870,625702
0,ALL,0.922518,0.061833,0.017182,0.975911,0.993803,3429,3717,226,16870,13170350


### Replace the output layer with nearest neighbour classifier

In [3]:
from sklearn.neighbors import KNeighborsClassifier

np.random.seed(123124)
seeds = list(map(int, np.random.random(10)*21321456))

final_res = pandas.DataFrame()
for SEED in seeds:
    print(SEED)
    # load CNN
    loaded_net = CNN()
    checkpoint = torch.load('All2All--CNN--{}'.format(SEED))
    loaded_net.load_state_dict(checkpoint['model_state_dict'])
    loaded_net.cuda()
    loaded_net.eval()
    
    # load data
    data_path = '../data/ACS_sources_original.csv'
    data = pandas.read_csv(data_path)
    galaxies_to_test = set([])

    available_galaxies = set(data['galaxy'].unique())
    available_galaxies = available_galaxies.difference(set(['VCC538']))
    images, labels, probabilities, galaxies, IDs = load_data(data, available_galaxies)

    # do the random split to recover correct training data
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    train_images, test_images, train_labels, test_labels = train_test_split(images, probabilities, test_size=TEST_SIZE, random_state=SEED)
    train_images, eval_images, train_labels, eval_labels = train_test_split(train_images, train_labels, test_size=EVAL_SIZE, random_state=SEED+50)

    test_labels = np.array(test_labels>=0.5, dtype=int)
    testData = CustomGCDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(testData, batch_size=BATCHSIZE, shuffle=False)

    train_labels = np.array(train_labels>=0.5, dtype=int)
    trainData = CustomGCDataset(train_images, train_labels)
    train_loader = torch.utils.data.DataLoader(trainData, batch_size=BATCHSIZE, shuffle=False)

    # get latent representation of training data
    new_train = []
    for x,y in train_loader:
        _, xout = loaded_net(x.cuda())
        xout = xout.cpu().detach().numpy()
        new_train += list(xout)

    # get latent representation of test data
    new_test = []
    for x,y in test_loader:
        _, xout = loaded_net(x.cuda())
        xout = xout.cpu().detach().numpy()
        new_test += list(xout)

    # train kNN classifier using the latent training data
    knn = KNeighborsClassifier(1)
    knn.fit(new_train, np.array(train_labels >= 0.5, dtype=int))
    # predict classes using the latent test data
    pred = knn.predict(new_test)
    probs = knn.predict_proba(new_test)[:,1]
    
    # evaluate performance metrics
    stats_gal, stats_all, _, _, _, _, _ = get_test_metrics(list(galaxies_to_test), [], [], test_labels, pred, probs=probs, thresh=thresh)
    final_res = final_res.append(stats_all)
final_res['seed'] = seeds 
final_res.to_csv('All2All--CNN-NN-results')

2780441


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


6488346


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


12269029


...

/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


625702


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


13170350


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [4]:
final_res.mean()

/tmp/ipykernel_816/3041093142.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_res.mean()


TPR             9.124109e-01
FDR             9.380693e-02
FPR             2.643293e-02
AUC(FDR,TPR)    8.961758e-01
AUC(FPR,TPR)    9.429890e-01
# found GCs     3.366000e+03
# total GCs     3.689200e+03
# fake GCs      3.484000e+02
# sources       1.687000e+04
seed            6.988626e+06
dtype: float64

In [6]:
final_res

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources,seed
0,ALL,0.915624,0.091149,0.026310,0.899180,0.944657,3440,3757,345,16870,2780441
0,ALL,0.917038,0.097534,0.026811,0.896559,0.945113,3294,3592,356,16870,6488346
0,ALL,0.909465,0.092030,0.025406,0.896035,0.942029,3315,3645,336,16870,12269029
0,ALL,0.914200,0.094242,0.026244,0.896978,0.943978,3335,3648,347,16870,2166833
0,ALL,0.908578,0.088235,0.024865,0.897695,0.941856,3379,3719,327,16870,6621300
0,ALL,0.901849,0.102300,0.028654,0.885513,0.936598,3317,3678,378,16870,13698722
0,ALL,0.915987,0.093624,0.026380,0.898130,0.944804,3369,3678,348,16870,2249589
0,ALL,0.920140,0.091585,0.026234,0.901328,0.946953,3422,3719,345,16870,9815943
0,ALL,0.905605,0.096068,0.027812,0.890889,0.938897,3425,3782,364,16870,625702
0,ALL,0.915623,0.091302,0.025614,0.899451,0.945005,3364,3674,338,16870,13170350


### Do the same using 12 nearest neighbours

In [7]:
np.random.seed(123124)
seeds = list(map(int, np.random.random(10)*21321456))

final_res = pandas.DataFrame()
for SEED in seeds:
    print(SEED)
    # load model
    loaded_net = CNN()
    checkpoint = torch.load('All2All--CNN--{}'.format(SEED))
    loaded_net.load_state_dict(checkpoint['model_state_dict'])
    loaded_net.cuda()
    loaded_net.eval()
    
    # load data
    data_path = '../data/ACS_sources_original.csv'
    data = pandas.read_csv(data_path)
    galaxies_to_test = set([])

    available_galaxies = set(data['galaxy'].unique())
    available_galaxies = available_galaxies.difference(set(['VCC538']))
    images, labels, probabilities, galaxies, IDs = load_data(data, available_galaxies)

    # create data split for random seed
    np.random.seed(SEED)
    torch.manual_seed(SEED)

    train_images, test_images, train_labels, test_labels = train_test_split(images, probabilities, test_size=TEST_SIZE, random_state=SEED)
    train_images, eval_images, train_labels, eval_labels = train_test_split(train_images, train_labels, test_size=EVAL_SIZE, random_state=SEED+50)

    test_labels = np.array(test_labels>=0.5, dtype=int)
    testData = CustomGCDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(testData, batch_size=BATCHSIZE, shuffle=False)

    train_labels = np.array(train_labels>=0.5, dtype=int)
    trainData = CustomGCDataset(train_images, train_labels)
    train_loader = torch.utils.data.DataLoader(trainData, batch_size=BATCHSIZE, shuffle=False)

    # get latent representation of training data
    new_train = []
    for x,y in train_loader:
        _, xout = loaded_net(x.cuda())
        xout = xout.cpu().detach().numpy()
        new_train += list(xout)

    # get latent representation of test data
    new_test = []
    for x,y in test_loader:
        _, xout = loaded_net(x.cuda())
        xout = xout.cpu().detach().numpy()
        new_test += list(xout)

    # fit a kNN model on the latent training data
    knn = KNeighborsClassifier(12)
    knn.fit(new_train, np.array(train_labels >= 0.5, dtype=int))
    # evaluate the model on the latent test data
    pred = knn.predict(new_test)
    probs = knn.predict_proba(new_test)[:,1]
    
    # get the evaluation metrics
    stats_gal, stats_all, _, _, _, _, _ = get_test_metrics(list(galaxies_to_test), [], [], test_labels, pred, probs=probs, thresh=thresh)
    final_res = final_res.append(stats_all)
final_res['seed'] = seeds 
final_res.to_csv('All2All--CNN-12NN-results')

2780441


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


6488346


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


12269029


...

/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


625702


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


13170350


/home/dodo/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
final_res.mean()

/tmp/ipykernel_816/3041093142.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  final_res.mean()


TPR             9.267570e-01
FDR             6.067661e-02
FPR             1.676059e-02
AUC(FDR,TPR)    9.703816e-01
AUC(FPR,TPR)    9.894951e-01
# found GCs     3.418800e+03
# total GCs     3.689200e+03
# fake GCs      2.209000e+02
# sources       1.687000e+04
seed            6.988626e+06
dtype: float64

In [10]:
final_res

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources,seed
0,ALL,0.928666,0.061087,0.017311,0.973096,0.990598,3489,3757,227,16870,2780441
0,ALL,0.936526,0.064776,0.017548,0.970781,0.989563,3364,3592,233,16870,6488346
0,ALL,0.924280,0.059726,0.016181,0.968850,0.989227,3369,3645,214,16870,12269029
0,ALL,0.932840,0.059165,0.016185,0.973089,0.990510,3403,3648,214,16870,2166833
0,ALL,0.923635,0.052936,0.014600,0.971504,0.988745,3435,3719,192,16870,6621300
0,ALL,0.917074,0.062013,0.016904,0.965611,0.987665,3373,3678,223,16870,13698722
0,ALL,0.936650,0.059771,0.016601,0.972947,0.991175,3445,3678,219,16870,2249589
0,ALL,0.924980,0.059852,0.016653,0.969966,0.988866,3440,3719,219,16870,9815943
0,ALL,0.917768,0.067687,0.019254,0.966438,0.988369,3471,3782,252,16870,625702
0,ALL,0.925150,0.059751,0.016369,0.971533,0.990235,3399,3674,216,16870,13170350
